# 关键概念

- 🌹 大模型
- 🌹 提示语
- 🌹 链
- 🌹 文本向量
- 🌹 检索
- 🌹 工具
- 🌹 智能体

## 大模型

In [63]:
from langchain_zhipu import ChatZhipuAI

In [64]:
chat = ChatZhipuAI()
chat.invoke("给我一个中文名字，三个字")

AIMessage(content='林子轩', response_metadata={'id': '8830400293413042041', 'created': 1720684997, 'token_usage': {'completion_tokens': 6, 'prompt_tokens': 13, 'total_tokens': 19}, 'model_name': 'glm-4-air', 'finish_reason': 'stop'}, id='run-765ab466-7aa7-442c-aaaa-5530331067d3-0')

In [67]:
for x in chat.stream("给我一个中文名字，三个字"):
    print(x.content, end="|")

林|晨|曦||

In [68]:
from langchain_community.llms import Tongyi
llm = Tongyi(model="qwen2-1.5b-instruct")
llm.invoke("给我一个中文名字，三个字")

'李晓明'

## 提示语

In [71]:
"abc"
who = "Adi"
f'hello {who} <f-string>'

'hello Adi <f-string>'

In [74]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("讲一个关于 {topic} 的笑话")
prompt_template.invoke("abc")
# prompt_template.input_schema.schema()
# prompt_template.invoke({"topic": "cats"})

StringPromptValue(text='讲一个关于 abc 的笑话')

In [81]:
prompt_template2 = PromptTemplate.from_template("告诉我 {topic} 的原理")

## Chain 链

In [75]:
chain = prompt_template | llm
chain.invoke({"topic": "汽车"})

'好的，这里有一个关于汽车的笑话：\n\n有一天，一辆汽车走进了修理厂，对修理工说：“我需要一个新的刹车。”\n修理工惊讶地问：“你会说话？”\n汽车回答：“是的，我刹车坏了。” \n\n希望这个笑话能让你会心一笑！'

In [84]:
chain = prompt_template2 | chat
for x in chain.stream({"topic": "汽车"}):
    print(x.content, end="")

汽车是一种通过内燃机或电动机产生动力，驱动车轮行驶的交通工具。下面我将分别介绍内燃机汽车和电动汽车的工作原理。

### 内燃机汽车原理

内燃机汽车是通过内燃机将燃料的化学能转化为机械能来驱动车辆。内燃机汽车的工作原理主要包括以下几个部分：

1. **发动机**：发动机是汽车的心脏，通过燃烧燃料产生动力。它的工作过程通常分为吸气、压缩、做功和排气四个冲程。
   
2. **燃料系统**：负责将燃料以正确的比例和压力输送到发动机燃烧室内。

3. **点火系统**：对于汽油机而言，点火系统负责在恰当的时刻产生火花，点燃燃料混合气。

4. **冷却系统**：通过循环水或冷却液吸收发动机产生的热量，防止发动机过热。

5. **润滑系统**：通过机油减少发动机内部运动部件之间的摩擦和磨损。

6. **传动系统**：包括离合器、变速箱、传动轴等，将发动机产生的动力传递到车轮。

7. **制动系统**：通过摩擦力减慢或停止车辆。

8. **转向系统**：控制车辆的行驶方向。

9. **悬挂系统**：连接车轮和车身，吸收路面冲击，保证行驶平顺。

### 电动汽车原理

电动汽车则是通过电动机将电池储存的电能转化为机械能来驱动车辆。电动汽车的工作原理主要包括以下几个部分：

1. **电池系统**：作为能量存储单元，提供电能给驱动电机。

2. **驱动电机系统**：将电能转化为机械能，驱动车轮转动。

3. **电控系统**：包括整车控制器、电机控制器等，负责调控电能的输出和电机的运转。

4. **充电系统**：为电池系统充电。

5. **辅助系统**：如DC/DC转换器、热管理系统等，保证车辆正常运行。

与内燃机汽车相比，电动汽车在运行过程中更加安静、清洁，且维护成本较低，但电池的续航里程和充电基础设施建设是其普及的主要限制因素。

无论是内燃机汽车还是电动汽车，它们都需要一个复杂的控制系统来确保车辆的安全、高效和舒适运行。

## 文本向量

In [85]:
from langchain_zhipu import ZhipuAIEmbeddings
embeddings_model = ZhipuAIEmbeddings()

In [86]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1024)

In [ ]:
# [-0.06281121,
#  0.05147552,
#  -0.016494857,
#  0.0023994024,
#  0.02036759,
#  0.009170211,
# ...]
embeddings[1]

In [ ]:
embedded_query = embeddings_model.embed_query(
    "What was the name mentioned in the conversation?")
embedded_query[:5]

## RAG 检索

In [89]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("txt/state_of_the_union.txt")
documents = loader.load()

from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

from langchain_community.vectorstores import FAISS
embeddings = ZhipuAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever()

In [91]:
docs = retriever.invoke("请告诉我关于乌克兰的核心观点")
docs

[Document(page_content='For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. \n\nAs I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  \n\nAnd we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  \n\nPutin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. \n\nAnd a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world.', metadata={'s

In [93]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("""
总统这样说：
----------------
{info}
----------------
请根据上述信息用中文回答我的问题：{task}
""")
p = prompt_template.invoke({
    "info": "We see the unity among leaders of nations and a more unified Europe a more unified West. And we see unity among the people who are gathering in cities in large crowds around the world even in Russia to demonstrate their support for Ukraine.  \n\nIn the battle between democracy and autocracy, democracies are rising to the moment, and the world is clearly choosing the side of peace and security. \n\nThis is a real test. It’s going to take time. So let us continue to draw inspiration from the iron will of the Ukrainian people. \n\nTo our fellow Ukrainian Americans who forge a deep bond that connects our two nations we stand with you.",
    "task": "请告诉我关于乌克兰的核心观点"
})
print(p.text)


总统这样说：
----------------
We see the unity among leaders of nations and a more unified Europe a more unified West. And we see unity among the people who are gathering in cities in large crowds around the world even in Russia to demonstrate their support for Ukraine.  

In the battle between democracy and autocracy, democracies are rising to the moment, and the world is clearly choosing the side of peace and security. 

This is a real test. It’s going to take time. So let us continue to draw inspiration from the iron will of the Ukrainian people. 

To our fellow Ukrainian Americans who forge a deep bond that connects our two nations we stand with you.
----------------
请根据上述信息用中文回答我的问题：请告诉我关于乌克兰的核心观点



In [94]:
def flat_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = {
    "info": retriever|flat_docs,
    "task": lambda y: y
} | prompt_template | llm

In [103]:
for x in chain.stream("请告诉我关于乌克兰的核心观点"):
    print(x, end="")

核心观点如下：

1. 国家领导人之间展现出团结，欧洲和西方整体趋向统一。
2. 世界各地的人民，包括在俄罗斯，通过大规模集会表达对乌克兰的支持。
3. 在民主与专制的斗争中，民主国家正在迎难而上，世界明显选择了和平与安全的立场。
4. 这是一场真正的考验，需要时间来解决。
5. 应该从乌克兰人民的坚强意志中汲取灵感。
6. 对于与乌克兰有着深厚联系的乌克兰裔美国人，表示坚定的支持。

In [105]:
chat.invoke("请问3219的平方是多少")

AIMessage(content='3219的平方是10371641。', response_metadata={'id': '8830405000697607042', 'created': 1720689859, 'token_usage': {'completion_tokens': 13, 'prompt_tokens': 12, 'total_tokens': 25}, 'model_name': 'glm-4-air', 'finish_reason': 'stop'}, id='run-c1791012-47ba-482e-8727-d7f0865a2765-0')

## 工具

In [106]:
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [add, multiply]

In [107]:
llm_with_tools = chat.bind(tools=[convert_to_openai_tool(t) for t in tools])

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'id': 'call_8830405619173014472', 'index': 0, 'type': 'function'}]}, response_metadata={'id': '8830405619173014472', 'created': 1720690050, 'token_usage': {'completion_tokens': 15, 'prompt_tokens': 270, 'total_tokens': 285}, 'model_name': 'glm-4-air', 'finish_reason': 'tool_calls'}, id='run-77dd961b-0b89-429d-8fff-f12008ae6996-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_8830405619173014472'}])

## Agent 智能体
[https://github.com/arcstep/langchain_zhipuai/blob/main/notes/agent.ipynb]